## Теоретическая часть

**1.** Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации.  
- Чем они отличаются и чем они похожи?  
- Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
- В чем принципиальные отличия item-item рекомендаций от ALS?


**Ответ 1:**  
Отличаются:  
- Назначением
  - Если нужно провести рекламную кампанию, то к товарам хорошо бы подобрать аудиторию
  - Если товар был куплен, то хорошо бы сделать предложение для кросспродажи (чехол для iPad)
- Размерностью (могут отличаться на порядки, например магазин устройств Apple)

Похожи:
- Сравнивают одинаковые сущности
- Квадратные, а значит одни и те же алгоритмы мат. обработки должны подходить обоим матрицам

**Ответ 2:**  
Вероятно функции рекомендаций могут быть использованы обеими матрицами, если внутри нет какой-то специальной для этого типа объектов обработки.

**Ответ 3:**  
ALS - позволяет сэкономить место путем разбиения матрицы users-items на 2 матрицы
item-item - нет разбиения матриц

Предположение:
item-item - для explicit систем
ALS - для implicit систем



**2.** Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


1. **total_discount = retail_disc + coupon_disc + coupon_match_disc** *(суммарный процент скидки)*
2. **total_discount_per_item = total_discount * sales_value** *(суммарная скидка на единицу товара)*
3. **total_sum_discount = total_discount \* quantity \* sales_value** *(суммарная скидка на весь приобретенный товар данного типа)*

**3.** Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


**Ответ**:  
1. Учитывает только факт покупки пользователем товара  
2. Требует больше времени/мощностей для вычислений  
3. Хуже интерпретируется чем item-item

**4.** Мы рассматривали bm_25_weight. 
Опишите, как он работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки). Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?

bm_25_weight использует параметр вхождения/невхождения. Больше, к сожалению, ничего не понял из статей. Гугление не помогло, надо разбирать на вебинаре.

[https://habr.com/ru/post/162937/]

[https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables]

## Практическая часть


In [1]:
import implicit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
import os, sys
from implicit.evaluation import precision_at_k
import warnings

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

warnings.filterwarnings('ignore')

In [2]:
def precision_at_k(recommended_list, bought_list, k=8):
    if type(bought_list) == str:
        bought_str_list = bought_list[1:-1].split(', ')
        bought_list = [int(item) for item in bought_str_list]

    if type(recommended_list) == str:
        bought_str_list = recommended_list[1:-1].split(', ')
        recommended_list = [int(item) for item in bought_str_list]

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)

    return precision

In [3]:
data = pd.read_csv('./data/retail_train.csv')

test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
result = pd.read_csv('./data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12695716, 8068934, 12428458, 434997, 858425]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13910611, 911878, 12301378, 9270394, 1109089]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.GridSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:  
1. Значение параметра - время обучения  
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!

In [17]:
items = pd.read_csv('./data/product.csv')
items.columns = [col.lower() for col in items.columns]
items.head(5)

,product_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [18]:
items.rename(columns={'product_id': 'item_id'}, inplace=True)
items.head(5)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [9]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [12]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0)
user_item_matrix = user_item_matrix.astype(float)
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values
matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))
id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))
itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

csr_mat = csr_matrix(user_item_matrix).T.tocsr()
bm25_weight_matrix = bm25_weight(user_item_matrix.T).T 
tfidf_matrix = tfidf_weight(user_item_matrix.T).T

In [17]:
mu = user_item_matrix.mean().mean()
ru = user_item_matrix.mean(axis=1)
cosine_matrix = (user_item_matrix.T - ru.values - mu).T

In [29]:
regularization_coeffs = [0.1, 1, 10, 100, 1000]
factors = [5, 10, 20, 40, 80]
matrixes = {
    'user_item': user_item_matrix,
    'bm25': bm25_weight_matrix,
    'tfidf': tfidf_matrix,
    'cosine': cosine_matrix
}

In [32]:
for ref_coef in regularization_coeffs:
    for factor in factors:
        for matrix in matrixes:
            model = AlternatingLeastSquares(
                factors=factor,
                regularization=ref_coef,
                iterations=10, 
                calculate_training_loss=True, 
                num_threads=4
            )
            model.fit(csr_matrix(matrixes[matrix]).T.tocsr(), show_progress=True)
            recs = model.recommend(
                userid=userid_to_id[2],
                user_items=csr_matrix(user_item_matrix).tocsr(),
                N=5, 
                filter_already_liked_items=False,
                filter_items=None, 
                recalculate_user=True
            )
            result[f'{matrix}-{factor}-{ref_coef}'] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])

100%|██████████| 10/10 [00:22<00:00,  2.29s/it, loss=0.874]


In [33]:
report = pd.DataFrame([
    (model, round(result.apply(lambda row: precision_at_k(recommended_list=row[str(model)], bought_list=row['actual']), axis=1).mean(), 5)) for model in list(result)[2:]
], columns=['The Name of the Model: ', 'precision@k'])

In [34]:
report.groupby('precision@k').get_group(report['precision@k'].max())

,The Name of the Model:,precision@k
0,user_item-5-0.1,0.17963
2,tfidf-5-0.1,0.17963
3,cosine-5-0.1,0.17963
4,user_item-10-0.1,0.17963
6,tfidf-10-0.1,0.17963
7,cosine-10-0.1,0.17963
11,cosine-20-0.1,0.17963
15,cosine-40-0.1,0.17963
20,user_item-5-1,0.17963
22,tfidf-5-1,0.17963
